In [9]:
# !pip install -U jupyter ipywidgets

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch

model_id = "google/codegemma-7b-it"
model_id = "codellama/CodeLlama-13b-Instruct-hf"
dtype = torch.bfloat16

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="cpu",
    # low_cpu_mem_usage=True,
    torch_dtype=dtype,
)

/home/eboros/.local/lib/python3.11/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
# Define the template function
def summarize_code(code: str, max_new_tokens: int) -> str:
    prompt = f"""
    [INST] <>
    You are an expert in Python Programming. Below is a line of python code that describes a task.
    Return only one line of summary that appropriately describes the task that the code is
    performing. You must write only summary without any prefix or suffix explanations.
    Note: The summary should have minimum 1 words and can have on an average 25 words.
    <>
    {code} [/INST]
    """
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Generate the summary
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # Decode the output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = summary.split("[/INST]")[-1].strip()
    return summary.strip()


In [15]:
# Define the template function
def summarize_code(code: str, max_new_tokens: int) -> str:
    prompt = f"""
    [INST] <>
    You are an expert in Python programming. Below is a Python code that describes a task. 
    Explain the code step by step with details about the implementation.
    <>
    {code} [/INST]
    """
    
    # Tokenize the input
    inputs = tokenizer(prompt, return_tensors="pt")
    
    # Generate the summary
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    
    # Decode the output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = summary.split("[/INST]")[-1].strip()
    return summary.strip()

In [4]:

# Example code to summarize
example_code = "import pandas as pd\n\ndf = pd.read_csv('data.csv')\nprint(df.head())"

# Get the summary
summary = summarize_code(example_code)
print(summary)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Reads data from a CSV file and displays the first few rows.


In [11]:
# Code example

def bubble_sort_string_list(string_list):
    n = len(string_list)
    for i in range(n):
        for j in range(0, n-i-1):
            if string_list[j] > string_list[j+1]:
                string_list[j], string_list[j+1] = string_list[j+1], string_list[j]
    return string_list

# Example usage
string_list = ["banana", "apple", "cherry", "date"]
sorted_list = bubble_sort_string_list(string_list)
print(sorted_list)


['apple', 'banana', 'cherry', 'date']


In [12]:
from utils import get_cell_with_comment

# Define the path to your notebook and the comment to search for
notebook_path = 'Code Explainer.ipynb'

# Get the cell containing the specific comment
cell = get_cell_with_comment(notebook_path)

cell

{'cell_type': 'code',
 'execution_count': 5,
 'id': '565747b7-0117-463b-b022-7f307751e5db',
 'metadata': {},
 'outputs': [{'name': 'stdout',
   'output_type': 'stream',
   'text': "['apple', 'banana', 'cherry', 'date']\n"}],
 'source': '# Code example\n\ndef bubble_sort_string_list(string_list):\n    n = len(string_list)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if string_list[j] > string_list[j+1]:\n                string_list[j], string_list[j+1] = string_list[j+1], string_list[j]\n    return string_list\n\n# Example usage\nstring_list = ["banana", "apple", "cherry", "date"]\nsorted_list = bubble_sort_string_list(string_list)\nprint(sorted_list)\n'}

In [16]:
if cell:
    print("Found the cell:")
    print(cell['source'])
else:
    print("No cell found with the specified comment.")

print(f'Generating explanation of code in cell {cell}.')
# Get the summary of the code
summary = summarize_code(cell['source'], max_new_tokens=256)

print('-'*50)
print(summary)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Found the cell:
# Code example

def bubble_sort_string_list(string_list):
    n = len(string_list)
    for i in range(n):
        for j in range(0, n-i-1):
            if string_list[j] > string_list[j+1]:
                string_list[j], string_list[j+1] = string_list[j+1], string_list[j]
    return string_list

# Example usage
string_list = ["banana", "apple", "cherry", "date"]
sorted_list = bubble_sort_string_list(string_list)
print(sorted_list)

Generating explanation of code in cell {'cell_type': 'code', 'execution_count': 5, 'id': '565747b7-0117-463b-b022-7f307751e5db', 'metadata': {}, 'outputs': [{'name': 'stdout', 'output_type': 'stream', 'text': "['apple', 'banana', 'cherry', 'date']\n"}], 'source': '# Code example\n\ndef bubble_sort_string_list(string_list):\n    n = len(string_list)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if string_list[j] > string_list[j+1]:\n                string_list[j], string_list[j+1] = string_list[j+1], string_list[j]\n